In [1]:
#!pip install elasticsearch

In [1]:
import elasticsearch
from elasticsearch import Elasticsearch
import pandas as pd
import json
from ast import literal_eval
from tqdm import tqdm
import datetime
import os
import sys
import numpy
from elasticsearch import helpers
print("loaded ...............")


loaded ...............


In [20]:
ENDPOINT="http://localhost:9200"

In [21]:
es = Elasticsearch(request_timeout=100, hosts=ENDPOINT)
es.ping()

/home/sahab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  


True

In [4]:
os.listdir()

['.ipynb_checkpoints', 'netflix_titles.csv', 'elastic-knn.ipynb']

In [5]:
df=pd.read_csv("netflix_titles.csv")
df.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...


In [6]:
titles=df["title"].to_list()
len(titles)

6234

In [9]:
import tensorflow as tf
import tensorflow_hub as hub

In [10]:
import numpy as np
module_url='https://tfhub.dev/google/nnlm-en-dim128/2'
embed = hub.KerasLayer(module_url)
vector = []
for c, title in enumerate(titles):
    x=tf.constant([title])
    embeddings=embed(x)
    x=np.asarray(embeddings)
    x=x[0].tolist()
    vector.append(x)

In [11]:
len(vector)

6234

In [12]:
iindex = 'netflix-titles'
requests = []
for i, doc in enumerate(titles):
    request = {}
    request["_op_type"] = "index"
    request["_index"] = iindex
    request["_id"] = i
    request["title"] = doc
    request["title_vector"] = vector[i]
    requests.append(request)

In [13]:
requests[0]

{'_op_type': 'index',
 '_index': 'netflix-titles',
 '_id': 0,
 'title': 'Norm of the North: King Sized Adventure',
 'title_vector': [0.08594008535146713,
  -0.09169774502515793,
  -0.08221834152936935,
  0.1603367030620575,
  0.05244443938136101,
  0.11267175525426865,
  -0.08382085710763931,
  0.09882048517465591,
  0.021728241816163063,
  -0.18144603073596954,
  0.0012927551288157701,
  0.030685216188430786,
  -0.04533662647008896,
  -0.07281119376420975,
  -0.11955680698156357,
  0.017013855278491974,
  0.033623743802309036,
  -0.009736377745866776,
  0.033763282001018524,
  0.1921098679304123,
  0.00620125001296401,
  0.015555042773485184,
  0.06574436277151108,
  0.11323074996471405,
  -0.10774068534374237,
  0.16938979923725128,
  -0.13922490179538727,
  -0.10309455543756485,
  -5.224486085353419e-05,
  0.023089049383997917,
  0.04559335485100746,
  -0.10510903596878052,
  -0.1005614772439003,
  -0.07881765812635422,
  0.025743374601006508,
  -0.05974612757563591,
  -0.1747056096

In [21]:
# settings = {
#     "settings":{
#         "number_of_shards":2,
#         "number_of_replicas":1,
#         "index.knn":True
#     },
#     "mappings":{
#         "dynamic":'true',
#         "_source":{
#             "enabled":'true'
#         },
#         "properties":{
#             "title":{
#               "type":"text"  
#             },
#             "title_vector":{
#                 "type":"knn_vector",
#                 "dimension":128               
                
#             }
            
#         }
        
#     }    
   
# }
knn_index = {
    "settings": {
        "index.knn": True
    },
    "mappings": {
        "properties": {
            "question_vector": {
                "type": "knn_vector",
                "dimension": 256
            }
        }
    }
}

In [14]:
mapping = {
    "dynamic": False,
    "properties": {
      "title": {
        "type": "text"
      },
      "title_vector": {
        "type": "knn_vector",
        "dimension": 128
      }
    }
  }


In [15]:
mapping

{'dynamic': False,
 'properties': {'title': {'type': 'text'},
  'title_vector': {'type': 'knn_vector', 'dimension': 128}}}

In [22]:
iindex = 'netflix-titles'
if es.indices.exists(index=iindex):
    es.indices.delete(index=iindex)
es.indices.create(index=iindex)
es.indices.put_mapping(index=iindex, meta=mapping)
es.indices.get_mapping(index=iindex)

/home/sahab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  
/home/sahab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/sahab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elas

ObjectApiResponse({'netflix-titles': {'mappings': {'_meta': {'dynamic': False, 'properties': {'title_vector': {'type': 'knn_vector', 'dimension': 128}, 'title': {'type': 'text'}}}}}})

In [53]:
help(es.indices.put_mapping)

Help on method put_mapping in module elasticsearch._sync.client.indices:

put_mapping(*, index: Union[str, List[str], Tuple[str, ...]], allow_no_indices: Union[bool, NoneType] = None, date_detection: Union[bool, NoneType] = None, dynamic: Union[ForwardRef("t.Literal['false', 'runtime', 'strict', 'true']"), str, NoneType] = None, dynamic_date_formats: Union[List[str], Tuple[str, ...], NoneType] = None, dynamic_templates: Union[Mapping[str, Mapping[str, Any]], List[Mapping[str, Mapping[str, Any]]], Tuple[Mapping[str, Mapping[str, Any]], ...], NoneType] = None, error_trace: Union[bool, NoneType] = None, expand_wildcards: Union[ForwardRef("t.Literal['all', 'closed', 'hidden', 'none', 'open']"), str, List[Union[ForwardRef("t.Literal['all', 'closed', 'hidden', 'none', 'open']"), str]], Tuple[Union[ForwardRef("t.Literal['all', 'closed', 'hidden', 'none', 'open']"), str], ...], NoneType] = None, field_names: Union[Mapping[str, Any], NoneType] = None, filter_path: Union[str, List[str], Tuple[st

In [23]:
indices=es.indices.get_alias().keys()
for Name in indices:
    print(Name)

my-approx-knn-index
my-exact-knn-index
netflix-titles
exampleindex


/home/sahab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  """Entry point for launching an IPython kernel.


In [24]:
try:
    res=helpers.bulk(es, requests, index=iindex)
    print("Working")
except Exception as e:
    print(e)




/home/sahab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  


Working


In [26]:
!curl -X GET "localhost:9200/{iindex}/_count"

{"count":6234,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}

In [27]:
title = "spiderman home coming"
x = tf.constant([title])
embeddings = embed(x)
x = np.asarray(embeddings)
x = x[0].tolist()



In [28]:
body= {
   "query":{
      "function_score":{
         "query":{
            "multi_match":{
               "query":title,
               "fields":[
                  "title"
               ]
            }
         },
         "boost_mode":"replace",
         "functions":  [
            
         ]
      }
   }
}

In [29]:

res = es.search(index=iindex, body=body, size=5)

for hit in res["hits"]["hits"]:
    print("id: {}, score: {}".format(hit["_id"], hit["_score"]))
    print(hit["_source"]['title'])
    print()

id: 1250, score: 6.615329
Iron Sky: The Coming Race

id: 1553, score: 6.261637
Away From Home

id: 4152, score: 6.261637
Home for Christmas

id: 1734, score: 5.517851
Take Me Home Tonight

id: 5200, score: 5.517851
Ram Dass, Going Home



/home/sahab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  """Entry point for launching an IPython kernel.
/home/sahab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  """Entry point for launching an IPython kernel.


In [30]:
#sample curl command
curl -X GET "localhost:9200/my-approx-knn-index/_knn_search?pretty" -H 'Content-Type: application/json' -d'
{
  "knn": {
    "field": "my-image-vector",
    "query_vector": [-0.5, 90.0, -10, 14.8, -156.0],
    "k": 10,
    "num_candidates": 100
  },
  "fields": [
    "my-image-vector",
    "my-tag"
  ]
}
'


SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-30-10f4a26da856>, line 13)

In [31]:
x

[-0.019000479951500893,
 0.11434528231620789,
 -0.05465194955468178,
 0.14108814299106598,
 -0.013230196200311184,
 -0.01774519868195057,
 -0.03944757953286171,
 -0.051330190151929855,
 0.010928218252956867,
 0.10952216386795044,
 0.03544875606894493,
 -0.06671733409166336,
 0.011308345943689346,
 -0.0351598784327507,
 -0.06555944681167603,
 -0.09359932690858841,
 -0.20417484641075134,
 -0.15182816982269287,
 -0.04365351423621178,
 -0.15484508872032166,
 0.04812793806195259,
 -0.08167213201522827,
 0.09944505989551544,
 0.06260702759027481,
 0.10921692848205566,
 -0.07084016501903534,
 0.06559886783361435,
 0.1389821618795395,
 -0.05537295714020729,
 -0.005390515550971031,
 0.05129179358482361,
 -0.06244787573814392,
 -0.046145278960466385,
 -0.002746670274063945,
 -0.06947936862707138,
 -0.06878000497817993,
 0.07911903411149979,
 0.16538846492767334,
 0.06931640207767487,
 0.06250498443841934,
 -0.058125048875808716,
 -0.024755995720624924,
 -0.001684224815107882,
 0.1063629388809204

In [39]:
!curl -X GET "localhost:9200/netflix-titles/_knn_search?pretty" -H 'Content-Type: application/json' -d'{  "knn": {    "field": "title-vector","query_vector": ,    "k": 10,    "num_candidates": 100 } "fields": ["title"  ]}'

{
  "error" : "Incorrect HTTP method for uri [/netflix-titles/_knn_search?pretty] and method [GET], allowed: [POST]",
  "status" : 405
}


In [98]:
# script_query = {
#     "script_score": {
#         "query": {"match_all": {}},
#         "script": {
#             "source": "cosineSimilarity(params.query_vector, doc['title_vector']) + 1.0",
#             "params": {"query_vector": x}
#         }
#     }
# }
script_query = {
    "knn": {
        "title_vector": {
            "vector": x,
            "k": 2
        }
    }
}

response = es.search(
    index=iindex,
    body={
        "size": 10,
        "query": script_query,
        "_source": {"includes": ["title", "body"]}
    }
)

for hit in response["hits"]["hits"]:
    print("id: {}, score: {}".format(hit["_id"], hit["_score"]))
    print(hit["_source"])
    print()

/home/sahab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.


BadRequestError: BadRequestError(400, 'parsing_exception', 'unknown query [knn]')